In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model="mistral-nemo:latest")

In [2]:
# """아래 코드는 처음 data를 vector db에 저장할 때"""
# import os
# import warnings
# warnings.filterwarnings("ignore")
# from langchain_community.document_loaders import DirectoryLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter

# loader = DirectoryLoader(path='test_con/input/news', glob='*.txt')

# data = loader.load()

# len(data)
# # 문서를 문장으로 분리
# ## 청크 크기 500, 각 청크의 50자씩 겹치도록 청크를 나눈다
# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=500,
#     chunk_overlap=50,
# )
# docs = text_splitter.split_documents(data)

# from langchain.embeddings import HuggingFaceEmbeddings

# # 문장을 임베딩으로 변환하고 벡터 저장소에 저장
# embeddings = HuggingFaceEmbeddings(
#     model_name='BAAI/bge-m3',
#     #model_kwargs={'device':'cpu'},
#     model_kwargs={'device':'cuda'},
#     encode_kwargs={'normalize_embeddings':True},
# )

# # 벡터 저장소 생성
# from langchain.vectorstores import Chroma
# vectorstore = Chroma.from_documents(docs, embeddings)


# # 벡터 저장소 경로 설정
# ## 현재 경로에 'vectorstore' 경로 생성
# vectorstore_path = 'vectorstore'
# os.makedirs(vectorstore_path, exist_ok=True)

# # 벡터 저장소 생성 및 저장
# vectorstore = Chroma.from_documents(docs, embeddings, persist_directory=vectorstore_path)
# # 벡터스토어 데이터를 디스크에 저장
# vectorstore.persist()
# print("Vectorstore created and persisted")

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

# 기존에 저장한 벡터 저장소 경로 설정
vectorstore_path = 'vectorstore'

# Chroma 벡터 저장소 불러오기
vectorstore = Chroma(
    persist_directory=vectorstore_path,
    embedding_function=HuggingFaceEmbeddings(
        model_name='BAAI/bge-m3',
        model_kwargs={'device':'cuda'},
        encode_kwargs={'normalize_embeddings':True},
    )
)

In [4]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

retriever = vectorstore.as_retriever(search_kwargs={'k': 3})

# 프롬프트 템플릿
template = '''모든 응답은 정확한 정보만을 바탕으로 제공되어야 합니다. 
응답에 필요한 정보가 벡터 저장소에 없을 경우, "모릅니다."라고 응답하세요.
한국어로 포괄적이고 명확한 답변을 제공해 주세요.

{context}

Question: {question}
'''

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

# RAG Chain 연결
rag_chain = (
    {'context': retriever | format_docs, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

In [6]:
# 벤치마킹에 사용할 질문 리스트
questions = [
    "주식 부정거래 의혹에 대한 뉴스기사에서 누가 보석으로 풀려났나?",
    "지방선거 구도를 다루는 기사에서 어떤 내용을 다루나?",
    
]

In [ ]:
# 벤치마킹 실행
results = []
for query in questions:
    answer = rag_chain.invoke(query)
    results.append((query, answer))
    print("Query:", query)
    print("Answer:", answer)
    print("-" * 50)


In [ ]:
# 결과 저장 (옵션)
with open("benchmark_results.txt", "w", encoding="utf-8") as f:
    for query, answer in results:
        f.write(f"Query: {query}\nAnswer: {answer}\n{'-'*50}\n")

print("벤치마킹이 완료되었습니다. 결과가 'benchmark_results_token500_50.txt'에 저장되었습니다.")

In [16]:
# Chain 실행
query = "“코로나19 동안 대한민국의 정치적 결정에 어떤 변화가 있었는가?”"
answer = rag_chain.invoke(query)

print("Query:", query)
print("Answer:", answer)

Query: “코로나19 동안 대한민국의 정치적 결정에 어떤 변화가 있었는가?”
Answer: "코로나19 기간 동안 대한민국의 정치적 결정을 볼 때, 정부는 사회적 거리두기 단계 조정 방안을 논의하고 결정해야 한다고 시사했으며, 이행과 실천, 그리고 현장의 수용성 등을 고려해 불합리하거나 실효성이 없는 조치는 과감히 수정하겠다고 강조했습니다. 또한 국민들의 인권침해 소지가 있는 무리한 방역대책을 결과적으로 사회 불안을 증폭시킨다는 점을 인식하고 있습니다."
